In [1]:
from collections import Counter
import numpy as np
import tensorflow.contrib.keras as kr
import tensorflow as tf
import time
from datetime import timedelta
import os
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"
trainDataPath = "s3://corpus-text-classification1/data/train_5500.label.txt"
testDataPath = "s3://corpus-text-classification1/data/TREC_10.label.txt"
savePath = "s3://corpus-2/model2/trec"

In [3]:
def loadGloVe(filename, emb_size=50):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    train_data = []
    with open(filePath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f.readlines():
            word = line.strip().split()
            label = word[0].split(":")[0]
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)

In [4]:
categories = ['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath, 100)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

cat_to_id = {'ABBR': 0, 'DESC': 1, 'ENTY': 2, 'HUM': 3, 'LOC': 4, 'NUM': 5}

len(embedding),embedding_dim,vocab_size

Loading GloVe!
Completed!


(400000, 100, 400000)

In [5]:
testData = readfile(testDataPath)
trainData = readfile(trainDataPath)

trainData = np.r_[trainData,testData]
len(trainData)

5952

In [6]:
seq_length = 0
for content in trainData[:,0]:
    if seq_length < len(content):
        seq_length = len(content)   # seq_length = 35
        
seq_length

37

In [7]:
def process_file(contents, labels, word_to_id, cat_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [8]:
print("Loading training and validation and testing data...")
start_time = time.time()
x_train, y_train = process_file(trainData[:,0], trainData[:,1], word_to_id,cat_to_id, num_classes, seq_length)
time_dif = get_time_dif(start_time)
print("Loading data Time usage:", time_dif)

Loading training and validation and testing data...
Loading data Time usage: 0:00:00


In [9]:
class rnn_cnn(object):
    
    def __init__(self, savePath, num_classes):
        self.savePath = savePath
        # 输入内容及对应的标签
        self.input_x = tf.placeholder(tf.int32, [None, seq_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
        # dropout的损失率
        self.keep_prob = tf.placeholder(tf.float64, name='keep_prob')
        # 词向量映射;实际上此处的词向量并不是用的预训练好的词向量，而是未经任何训练直接生成了一个矩阵，将此矩阵作为词向量矩阵使用，效果也还不错。
        # 若使用训练好的词向量，或许训练此次文本分类的模型时会更快，更好。
        # embedding = tf.get_variable('embedding', [vocab_size, embedding_dim])
        embedding_inputs = tf.nn.embedding_lookup(embedding, self.input_x)

        num_filters = 256
        kernel_size = 5
        hidden_dim = 128
        learning_rate = 1e-3

        # CNN layer
        conv = tf.layers.conv1d(embedding_inputs, num_filters, kernel_size, name='conv')  # num_filters = 256 这是个啥
        ''' https://blog.csdn.net/khy19940520/article/details/89934335
        tf.layers.conv1d：一维卷积一般用于处理文本数据，常用语自然语言处理中，输入一般是文本经过embedding的二维数据。
            inputs： 输入tensor， 维度(batch_size, seq_length, embedding_dim) 是一个三维的tensor；其中，
                batch_size指每次输入的文本数量；
                seq_length指每个文本的词语数或者单字数；
                embedding_dim指每个词语或者每个字的向量长度；
                例如每次训练输入2篇文本，每篇文本有100个词，每个词的向量长度为20，那input维度即为(2, 100, 20)。
            filters：过滤器（卷积核）的数目
            kernel_size：卷积核的大小，卷积核本身应该是二维的，这里只需要指定一维，因为第二个维度即长度与词向量的长度一致，卷积核只能从上往下走，不能从左往右走，即只能按照文本中词的顺序，也是列的顺序。
        '''
        # global max pooling layer
        gmp = tf.reduce_max(conv, reduction_indices=[1], name='gmp')  # https://blog.csdn.net/lllxxq141592654/article/details/85345864

        # 全连接层，后面接dropout以及relu激活
        fc = tf.layers.dense(gmp, hidden_dim, name='fc1')  # hidden_dim：128
        ''' https://blog.csdn.net/yangfengling1023/article/details/81774580
        dense ：全连接层  inputs：输入该网络层的数据；units：输出的维度大小，改变inputs的最后一维
        '''
        fc = tf.nn.dropout(fc, self.keep_prob)
        fc = tf.nn.relu(fc)

        # 分类器
        logits = tf.layers.dense(fc, num_classes, name='fc2')
        self.y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）

        # 损失函数，交叉熵
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.input_y)
        self.loss = tf.reduce_mean(cross_entropy)
        # 优化器
        self.optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

        # 准确率
        correct_pred = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)
        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        self.saver = tf.train.Saver()
        
    
    def train(self, x_train, y_train, split_type, fold_id, num_epochs=20, dropout_keep_prob=0.5, print_per_batch=30, batch_size=64):
        
        savePath = "%s/%s/%s/%s" % (self.savePath, split_type, fold_id, fold_id)
        # 创建session
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        
        print('Training and evaluating...')
        start_time = time.time()
        total_batch = 0  # 总批次
        best_acc_train = 0.0  # 最佳验证集准确率
        last_improved = 0  # 记录上一次提升批次
        require_improvement = 500  # 如果超过1000轮未提升，提前结束训练
        flag = False

        for epoch in range(num_epochs):  # 20
            print('Epoch:', epoch + 1)
            batch_train = rnn_cnn.batch_iter(x_train, y_train, batch_size)
            for x_batch, y_batch in batch_train:
                feed_dict = {self.input_x: x_batch, self.input_y: y_batch, self.keep_prob: dropout_keep_prob}
                session.run(self.optim, feed_dict=feed_dict)  # 运行优化
                total_batch += 1

                if total_batch % print_per_batch == 0:
                    # 每多少轮次输出在训练集和验证集上的性能
                    feed_dict[self.keep_prob] = 1.0
                    loss_train, acc_train = session.run([self.loss, self.acc], feed_dict=feed_dict)
                    # loss_val, acc_val = evaluate(session, x_dev, y_dev, loss, acc)
                    if acc_train > best_acc_train:
                        # 保存最好结果
                        best_acc_train = acc_train
                        last_improved = total_batch
                        self.saver.save(sess=session, save_path=savePath)
                        improved_str = '*'
                    else:
                        improved_str = ''

                    time_dif = get_time_dif(start_time)
                    msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%}, Time: {3} {4}'
                    print(msg.format(total_batch, loss_train, acc_train, time_dif, improved_str))

                if total_batch - last_improved > require_improvement:
                    # 验证集正确率长期不提升，提前结束训练
                    print("No optimization for a long time, auto-stopping...")
                    flag = True
                    break  # 跳出循环
            if flag:  # 同上
                break
                
        session.close()
        return best_acc_train
        
        
    def evaluate_model(self, x_test, y_test, split_type, fold_id, categories, batch_size=64):
        
        savePath = "%s/%s/%s/%s" % (self.savePath, split_type, fold_id, fold_id)
        # 读取保存的模型
        session = tf.Session()
        self.saver.restore(sess=session, save_path=savePath)
        start_time = time.time()
        print('Testing...')
        loss_test, acc_test = self.evaluate(session, x_test, y_test, self.loss, self.acc, batch_size)
        msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
        print(msg.format(loss_test, acc_test))

        test_data_len = len(x_test)
        test_num_batch = int((test_data_len - 1) / batch_size) + 1

        y_test_cls = np.argmax(y_test, 1)  # 获得类别
        y_test_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

        for i in range(test_num_batch):  # 逐批次处理
            start_id = i * batch_size
            end_id = min((i + 1) * batch_size, test_data_len)
            feed_dict = {
                self.input_x: x_test[start_id:end_id],
                self.keep_prob: 1.0
            }
            y_test_pred_cls[start_id:end_id] = session.run(self.y_pred_cls, feed_dict=feed_dict)

        # 评估
        print("Precision, Recall and F1-Score...")
        print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
        '''
        sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
            y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
            y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
            labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
            target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
            原文链接：https://blog.csdn.net/akadiao/article/details/78788864
        '''

        # 混淆矩阵
        print("Confusion Matrix...")
        cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
        '''
        混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
        这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
        https://blog.csdn.net/u011734144/article/details/80277225
        '''
        print(cm)

        time_dif = get_time_dif(start_time)
        print("Time usage:", time_dif)
        session.close()
        
        return acc_test
        
        
    def predict(self, predict_sentences, word_to_id, pad_max_length, split_type, fold_id):
        """
        将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
        """
        savePath = "%s/%s/%s/%s" % (self.savePath, split_type, fold_id, fold_id)
        session = tf.Session()
        self.saver.restore(sess=session, save_path=savePath)
        
        data_id = []
        # 将文本内容转换为对应的id形式
        for i in range(len(predict_sentences)):
            data_id.append([word_to_id[x] for x in predict_sentences[i].lower().strip().split() if x in word_to_id])

        # 使用keras提供的pad_sequences来将文本pad为固定长度
        x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
        ''' https://blog.csdn.net/TH_NUM/article/details/80904900
        pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
            sequences：浮点数或整数构成的两层嵌套列表
            maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
            dtype：返回的numpy array的数据类型
            padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
            truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
            value：浮点数，此值将在填充时代替默认的填充值0
        '''
        feed_dict = {
            self.input_x: x_pad,
            self.keep_prob: 1.0
        }
        predict_result = session.run(self.y_pred_cls, feed_dict=feed_dict)
        predict_result = [i+1 for i in predict_result]
        session.close()
        
        return predict_result
    
    
    def evaluate(self, sess, x_pad, y_pad, loss1, acc1, batch_size):
        """评估在某一数据上的准确率和损失"""
        data_len = len(x_pad)
        batch_eval = rnn_cnn.batch_iter(x_pad, y_pad, batch_size)  
        total_loss = 0.0
        total_acc = 0.0
        # print(dropout_keep_prob)
        for x_batch1, y_batch1 in batch_eval:
            batch_len = len(x_batch1)
            feed_dict1 = {self.input_x: x_batch1, self.input_y: y_batch1, self.keep_prob: 1.0}
            lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
            total_loss += lossTmp * batch_len
            total_acc += accTmp * batch_len

        return total_loss / data_len, total_acc / data_len
    
    
    def batch_iter(x_pad, y_pad, batch_size=64):  # 128
        """生成批次数据"""
        data_len = len(x_pad)
        num_batch = int((data_len - 1) / batch_size) + 1
        # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
        # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
        indices = np.random.permutation(np.arange(data_len))
        x_shuffle = x_pad[indices]
        y_shuffle = y_pad[indices]

        # 返回所有batch的数据
        for i in range(num_batch):
            start_id = i * batch_size
            end_id = min((i + 1) * batch_size, data_len)
            yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
            
            

In [10]:
kf = KFold(n_splits=10)
model = rnn_cnn(savePath, 6)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
fold_id = 0
model_train_acc = []
model_test_acc = []
for train_i, test_i in kf.split(x_train):
    fold_id += 1
    print("Fold: ", fold_id)
    model_train_acc.append(model.train(x_train[train_i], y_train[train_i],"cnn",fold_id))
    model_test_acc.append(model.evaluate_model(x_train[test_i], y_train[test_i],"cnn",fold_id,categories))

Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.3, Train Acc:  48.44%, Time: 0:00:07 *
Iter:     60, Train Loss:    1.2, Train Acc:  48.44%, Time: 0:00:08 
Epoch: 2
Iter:     90, Train Loss:   0.85, Train Acc:  68.75%, Time: 0:00:13 *
Iter:    120, Train Loss:   0.75, Train Acc:  70.31%, Time: 0:00:19 *
Iter:    150, Train Loss:   0.72, Train Acc:  76.56%, Time: 0:00:25 *
Epoch: 3
Iter:    180, Train Loss:    0.7, Train Acc:  76.56%, Time: 0:00:26 
Iter:    210, Train Loss:   0.76, Train Acc:  71.88%, Time: 0:00:26 
Iter:    240, Train Loss:    0.5, Train Acc:  84.38%, Time: 0:00:32 *
Epoch: 4
Iter:    270, Train Loss:   0.43, Train Acc:  85.94%, Time: 0:00:39 *
Iter:    300, Train Loss:   0.47, Train Acc:  85.94%, Time: 0:00:39 
Iter:    330, Train Loss:   0.34, Train Acc:  85.94%, Time: 0:00:40 
Epoch: 5
Iter:    360, Train Loss:   0.51, Train Acc:  84.38%, Time: 0:00:40 
Iter:    390, Train Loss:   0.26, Train Acc:  96.88%, Time: 0:00:46 *
Iter:    420, T

INFO:tensorflow:Restoring parameters from s3://corpus-2/model2/trec/cnn/1/1


Testing...
Test Loss:   0.93, Test Acc:  73.83%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       1.00      0.40      0.57        10
        DESC       0.65      0.77      0.70       126
        ENTY       0.60      0.79      0.68       136
         HUM       0.88      0.71      0.79       137
         LOC       0.82      0.70      0.76        88
         NUM       0.94      0.74      0.82        99

   micro avg       0.74      0.74      0.74       596
   macro avg       0.81      0.68      0.72       596
weighted avg       0.77      0.74      0.74       596

Confusion Matrix...
[[  4   6   0   0   0   0]
 [  0  97  20   3   5   1]
 [  0  19 107   6   4   0]
 [  0   9  26  97   2   3]
 [  0  12  11   2  62   1]
 [  0   7  14   2   3  73]]
Time usage: 0:00:02
Fold:  2
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.3, Train Acc:  48.44%, Time: 0:00:07 *
Iter:     60, Train Loss:    1.2, Train Acc:  50.00%, Tim

INFO:tensorflow:Restoring parameters from s3://corpus-2/model2/trec/cnn/2/2


Testing...
Test Loss:   0.89, Test Acc:  76.85%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       1.00      0.67      0.80         6
        DESC       0.71      0.77      0.74       138
        ENTY       0.70      0.76      0.73       139
         HUM       0.84      0.73      0.78       147
         LOC       0.74      0.84      0.78        80
         NUM       0.93      0.79      0.86        86

   micro avg       0.77      0.77      0.77       596
   macro avg       0.82      0.76      0.78       596
weighted avg       0.78      0.77      0.77       596

Confusion Matrix...
[[  4   1   0   0   1   0]
 [  0 106  15   6  10   1]
 [  0  17 105  10   4   3]
 [  0  11  22 108   6   0]
 [  0   9   2   1  67   1]
 [  0   5   6   4   3  68]]
Time usage: 0:00:02
Fold:  3
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.3, Train Acc:  51.56%, Time: 0:00:07 *
Iter:     60, Train Loss:    1.1, Train Acc:  53.12%, Tim

...

Testing...
Test Loss:   0.65, Test Acc:  80.84%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       0.88      0.64      0.74        11
        DESC       0.79      0.69      0.74       135
        ENTY       0.75      0.81      0.78       135
         HUM       0.79      0.90      0.84       124
         LOC       0.81      0.84      0.83        93
         NUM       0.95      0.85      0.90        97

   micro avg       0.81      0.81      0.81       595
   macro avg       0.83      0.79      0.80       595
weighted avg       0.81      0.81      0.81       595

Confusion Matrix...
[[  7   2   2   0   0   0]
 [  1  93  21   8   9   3]
 [  0   7 110  15   2   1]
 [  0   4   6 111   3   0]
 [  0   7   4   4  78   0]
 [  0   5   4   2   4  82]]
Time usage: 0:00:02
Fold:  9
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.2, Train Acc:  62.50%, Time: 0:00:08 *
Iter:     60, Train Loss:    1.1, Train Acc:  57.81%, Tim

INFO:tensorflow:Restoring parameters from s3://corpus-2/model2/trec/cnn/9/9


Testing...
Test Loss:   0.74, Test Acc:  77.31%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       1.00      0.71      0.83        14
        DESC       0.79      0.63      0.70       131
        ENTY       0.78      0.74      0.76       141
         HUM       0.75      0.88      0.81       120
         LOC       0.75      0.79      0.77        97
         NUM       0.79      0.86      0.82        92

   micro avg       0.77      0.77      0.77       595
   macro avg       0.81      0.77      0.78       595
weighted avg       0.78      0.77      0.77       595

Confusion Matrix...
[[ 10   2   2   0   0   0]
 [  0  83  17   9  12  10]
 [  0   7 105  17   6   6]
 [  0   3   7 106   2   2]
 [  0   8   2   7  77   3]
 [  0   2   2   3   6  79]]
Time usage: 0:00:02
Fold:  10
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.5, Train Acc:  50.00%, Time: 0:00:09 *
Iter:     60, Train Loss:    1.1, Train Acc:  65.62%, Ti

INFO:tensorflow:Restoring parameters from s3://corpus-2/model2/trec/cnn/10/10


Testing...
Test Loss:   0.65, Test Acc:  81.18%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       1.00      0.80      0.89        10
        DESC       0.69      0.96      0.80       157
        ENTY       0.86      0.68      0.76       116
         HUM       0.90      0.86      0.88        85
         LOC       0.83      0.63      0.72        98
         NUM       0.91      0.86      0.88       129

   micro avg       0.81      0.81      0.81       595
   macro avg       0.86      0.80      0.82       595
weighted avg       0.83      0.81      0.81       595

Confusion Matrix...
[[  8   2   0   0   0   0]
 [  0 150   4   0   2   1]
 [  0  26  79   4   4   3]
 [  0   6   2  73   1   3]
 [  0  26   4   2  62   4]
 [  0   7   3   2   6 111]]
Time usage: 0:00:02


In [13]:
model_test_acc

[0.738255033557047,
 0.7684563766389885,
 0.7680672272914598,
 0.7865546227503224,
 0.8050420177083055,
 0.7512605046023841,
 0.7596638652456909,
 0.8084033608436585,
 0.7731092445990618,
 0.8117647060827047,
 0.7567114085959108]

In [14]:
np.mean(model_test_acc),np.std(model_test_acc),np.std(model_test_acc,ddof=1),np.var(model_test_acc)

(0.7752080334468666,
 0.023511829348229672,
 0.024659414657089925,
 0.0005528061193002741)

In [12]:
split_type = "random"
for train_i, test_i in kf.split(x_train):
    fold_id += 1
    print("Fold: ", fold_id)
    model_train_acc.append(model.train(x_train[train_i], y_train[train_i],split_type,fold_id))
    model_test_acc.append(model.evaluate_model(x_train[test_i], y_train[test_i],split_type,fold_id,categories))

Fold:  11
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.4, Train Acc:  45.31%, Time: 0:00:08 *
Iter:     60, Train Loss:    1.2, Train Acc:  54.69%, Time: 0:00:13 *
Epoch: 2
Iter:     90, Train Loss:    1.1, Train Acc:  60.94%, Time: 0:00:19 *
Iter:    120, Train Loss:   0.77, Train Acc:  75.00%, Time: 0:00:24 *
Iter:    150, Train Loss:   0.81, Train Acc:  76.56%, Time: 0:00:30 *
Epoch: 3
Iter:    180, Train Loss:   0.65, Train Acc:  78.12%, Time: 0:00:36 *
Iter:    210, Train Loss:   0.68, Train Acc:  71.88%, Time: 0:00:36 
Iter:    240, Train Loss:   0.58, Train Acc:  71.88%, Time: 0:00:37 
Epoch: 4
Iter:    270, Train Loss:   0.46, Train Acc:  85.94%, Time: 0:00:43 *
Iter:    300, Train Loss:    0.5, Train Acc:  79.69%, Time: 0:00:43 
Iter:    330, Train Loss:   0.42, Train Acc:  84.38%, Time: 0:00:44 
Epoch: 5
Iter:    360, Train Loss:   0.41, Train Acc:  85.94%, Time: 0:00:45 
Iter:    390, Train Loss:   0.27, Train Acc:  92.19%, Time: 0:00:50 *
Iter:    420,

INFO:tensorflow:Restoring parameters from s3://corpus-2/model2/trec/random/11/11


Testing...
Test Loss:   0.92, Test Acc:  75.67%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       0.60      0.60      0.60        10
        DESC       0.65      0.79      0.71       126
        ENTY       0.70      0.73      0.71       136
         HUM       0.84      0.74      0.79       137
         LOC       0.80      0.72      0.75        88
         NUM       0.90      0.83      0.86        99

   micro avg       0.76      0.76      0.76       596
   macro avg       0.75      0.73      0.74       596
weighted avg       0.77      0.76      0.76       596

Confusion Matrix...
[[  6   3   1   0   0   0]
 [  4  99  13   5   4   1]
 [  0  21  99  10   4   2]
 [  0  10  18 102   4   3]
 [  0  13   5   4  63   3]
 [  0   6   6   1   4  82]]
Time usage: 0:00:02
Fold:  12
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.4, Train Acc:  31.25%, Time: 0:00:07 *
Iter:     60, Train Loss:    1.2, Train Acc:  54.69%, Ti

SystemError: <built-in function WritableFile_Close> returned a result with an error set

In [ ]:
model_test_acc

In [ ]:
np.mean(model_test_acc),np.std(model_test_acc),np.std(model_test_acc,ddof=1),np.var(model_test_acc)

In [ ]:
def expert_split():
    train_data = [[] for i in range(20)]
    with open(trainDataPath, "r", encoding='utf-8') as fp:
        for line in fp.readlines():
            word = line.split()
            info = word[0]
            index = int(info.split(":")[4]) - 1
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data[index].append([content,label])
            
    for i in range(20):
        np.random.shuffle(train_data[i])
        train_data[i] = np.asarray(train_data[i])
        
    np.random.shuffle(train_data)
        
    return np.asarray(train_data)

In [ ]:
train_data_expert = expert_split()

In [ ]:
train_data_expert[0][:2]

In [ ]:
len(train_data_expert)

In [ ]:
expert_train_acc = []
expert_test_acc = []
split_type = "expert"
fold_id = 0
for train_i, test_i in kf.split(x_train):
    fold_id += 1
    print("Fold: ", fold_id)
    expert_train_acc.append(model.train(x_train[train_i], y_train[train_i],split_type,fold_id))
    expert_test_acc.append(model.evaluate_model(x_train[test_i], y_train[test_i],split_type,fold_id,categories))

In [ ]:
expert_test_acc

In [ ]:
np.mean(expert_test_acc),np.std(expert_test_acc),np.std(expert_test_acc,ddof=1),np.var(expert_test_acc)

In [ ]:
ex_train = []
ey_train = []
for ti in train_data_expert:
    x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
    ex_train.append(x_train)
    ey_train.append(y_train)

In [ ]:
ex_train = np.asarray(ex_train)
ey_train = np.asarray(ey_train)

In [ ]:
len(ex_train[0]),len(ex_train[1]),len(ex_train[0][0]),len(ex_train)

In [ ]:
def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y

In [ ]:
expert_train_acc = []
expert_test_acc = []
split_type = "expert"
fold_id = 0
for train_i, test_i in kf.split(ex_train):
    fold_id += 1
    print("Fold: ", fold_id)
    train_x, train_y = mergeData(ex_train[train_i],ey_train[train_i])
    test_x, test_y = mergeData(ex_train[test_i],ey_train[test_i])
    expert_train_acc.append(model.train(train_x, train_y,split_type,fold_id))
    expert_test_acc.append(model.evaluate_model(test_x, test_y,split_type,fold_id,categories))

In [ ]:
expert_test_acc

In [ ]:
np.mean(expert_test_acc),np.std(expert_test_acc),np.std(expert_test_acc,ddof=1),np.var(expert_test_acc)

In [ ]:
def bundle_split():
    train_data = [[] for i in range(920)]
    with open(trainDataPath, "r", encoding='utf-8') as fp:
        for line in fp.readlines():
            word = line.split()
            info = word[0].split(":")
            index = int(info[1]) - 1
            label = int(info[0])
            content = word[1:]
            train_data[index].append([content,label])
            
    for i in range(920):
        np.random.shuffle(train_data[i])
        train_data[i] = np.asarray(train_data[i])
        
    np.random.shuffle(train_data)   
    
    return train_data

train_data_bundle = bundle_split()

In [ ]:
train_data_bundle[0][:5,1]

In [ ]:
def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, split_type):
    
    print(split_type)
    
    tx = []
    ty = []
    for ti in train_data:
        x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
        tx.append(x_train)
        ty.append(y_train)

    tx = np.asarray(tx)
    ty = np.asarray(ty)
    
    print(len(tx),len(tx[0]),len(tx[1]),len(tx[0][0]))
    
    train_acc = []
    test_acc = []
    fold_id = 0
    
    for train_i, test_i in kf.split(tx):
        fold_id += 1
        print("Fold: ", fold_id)
        train_x, train_y = mergeData(tx[train_i],ty[train_i])
        test_x, test_y = mergeData(tx[test_i],ty[test_i])
        train_acc.append(model.train(train_x, train_y,split_type,fold_id))
        test_acc.append(model.evaluate_model(test_x, test_y,split_type,fold_id,categories))
        
    print(test_acc)
    print(np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc))
    
    return test_acc

In [ ]:
bundle_test_acc = train_split_data(train_data_bundle, "bundle")

In [ ]:
def table_split():
    train_data = [[] for i in range(37)]
    with open(trainDataPath, "r", encoding='utf-8') as fp:
        for line in fp.readlines():
            word = line.split()
            info = word[0].split(":")
            index = int(info[3]) - 1
            label = int(info[0])
            content = word[1:]
            train_data[index].append([content,label])
            
    for i in range(37):
        np.random.shuffle(train_data[i])
        train_data[i] = np.asarray(train_data[i])
        
    np.random.shuffle(train_data)   
    
    return train_data

train_data_table = table_split()

In [ ]:
table_test_acc = train_split_data(train_data_table, "table")

In [ ]:
expert_train_acc = []
expert_test_acc = []
split_type = "expert"
fold_id = 0
for train_i, test_i in kf.split(ex_train):
    fold_id += 1
    print("Fold: ", fold_id)
    train_x, train_y = mergeData(ex_train[train_i],ey_train[train_i])
    test_x, test_y = mergeData(ex_train[test_i],ey_train[test_i])
    expert_train_acc.append(model.train(train_x, train_y,split_type,fold_id))
    expert_test_acc.append(model.evaluate_model(test_x, test_y,split_type,fold_id,categories))


bundle_train_acc = []
bundle_test_acc = []
split_type = "bundle"
fold_id = 0
for train_i, test_i in kf.split(bx_train):
    fold_id += 1
    print("Fold: ", fold_id)
    expert_train_acc.append(model.train(x_train[train_i], y_train[train_i],split_type,fold_id))
    expert_test_acc.append(model.evaluate_model(x_train[test_i], y_train[test_i],split_type,fold_id,categories))

In [ ]:
np.mean(model_test_acc),np.std(model_test_acc),np.std(model_test_acc,ddof=1),np.var(model_test_acc)